In [14]:
import polars as pl

In [15]:
similarity_weight = 0.3
court_weight = 0.3
case_weight = 0.4
df = pl.read_csv("./cases.csv")

In [16]:
num_docs = 5

In [ ]:
df.with_columns(
    (pl.col("score") - pl.col("score").min())/((pl.col("score").max() - pl.col("score").min())),
    (pl.col("CasePageRank") - pl.col("CasePageRank").min())/((pl.col("CasePageRank").max() - pl.col("CasePageRank").min())),
    (pl.col("CourtPageRank") - pl.col("CourtPageRank").min())/((pl.col("CourtPageRank").max() - pl.col("CourtPageRank").min()))
).with_columns(
    pl.struct(["score", "CasePageRank", "CourtPageRank"]).map_elements(lambda x: similarity_weight*x["score"] + court_weight*x["CourtPageRank"] + case_weight*x["CasePageRank"]).alias("FinalScore")
).group_by(["Case", "FiledDate", "CourtName"]).max().sort("FinalScore", descending = True).top_k(num_docs, by = "FinalScore")

Case,FiledDate,CourtName,score,Summary,CasePageRank,CourtPageRank,FinalScore
str,str,str,f64,str,f64,f64,f64
"""Meeropol v. Ni…","""1977/07/28""","""Court of Appea…",0.251665,"""This legal opi…",1.0,0.416667,0.6005
"""Disney Enterpr…","""2019/03/06""","""Court of Appea…",0.751472,"""This legal opi…",0.037756,1.0,0.540544
"""Authors Guild …","""2015/10/16""","""Court of Appea…",1.0,"""This legal opi…",0.26577,0.416667,0.531308
"""Cambridge Univ…","""2014/10/17""","""Court of Appea…",0.550994,"""This legal opi…",0.044086,0.75,0.407933
"""De Fontbrune v…","""2022/07/13""","""Court of Appea…",0.319278,"""## Summary of …",0.005074,1.0,0.397813
